<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Test-connectivity" data-toc-modified-id="Test-connectivity-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Test connectivity</a></span></li><li><span><a href="#Test-watching-legimens.Object" data-toc-modified-id="Test-watching-legimens.Object-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test watching legimens.Object</a></span></li><li><span><a href="#Changing-sys.switchinterval" data-toc-modified-id="Changing-sys.switchinterval-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Changing sys.switchinterval</a></span></li><li><span><a href="#Simple-object-watch" data-toc-modified-id="Simple-object-watch-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Simple object watch</a></span></li></ul></div>

In [1]:
from libvis import Vis
import libvis
import json
import numpy as np

from legimens.Object import ref
from legimens import Object

%load_ext autoreload
%autoreload 2

No module named 'bokeh'


In [2]:
vis = Vis(ws_port=7700, vis_port=7000)
vis.start()

Starting http at 7000
2020-05-15 02:02:19.171 | INFO     | legimens.websocket.server:start_server:33 - Starting ws server at localhost:7700


## Test connectivity

In [3]:
vis.vars.image = 1


## Test watching legimens.Object


In [4]:

class Var(libvis.modules.BaseModule):
    def _prepare_send(self, name, value):
        return name, {'type':'Object','value':value}

x = Var(x=1)
vis.app.watch_obj(x)
vis.vars.x = x

2020-05-15 02:02:30.758 | INFO     | legimens.App:_handler:74 - New ws connection of /Legi_0x7f30399864a0 from ::1
2020-05-15 02:02:30.765 | INFO     | legimens.App:_handle_obj_ref:105 - Yield initial update {"x": {"type": "Object", "value": 1}}


In [5]:
for i in range(100000000//3):
    x['x'] = i

In [ ]:
vis.app._watch_poll_delay = .1
type(vis.app.vars)

## Changing sys.switchinterval 
determines how often we switch between threads to send updates

Try to set larger value and run previous tests

In [ ]:
import sys
sys.getswitchinterval()

In [ ]:
sys.setswitchinterval(0.04)

In [ ]:
vis.vars.foo=2

## Simple object watch

Note that CPython has a limitation in it's implementation of mutable objects:
it can't create a weakref for base types.
This is fixed for mutables in those additional classes

For more info check: https://mail.python.org/pipermail/python-list/2005-March/346298.html

In [6]:
class List(list):
    name='VisVar'
    pass

libvis.interface.add_serializer(List, ref)
def ser_type(value):
    return json.dumps({'value':value})
    return str(value)
vis.app.serialize_value = ser_type

li = List()



In [7]:
vis.app.watch_obj(li)
vis.vars.x = li

2020-05-15 02:02:41.257 | ERROR    | legimens.App:_handler:89 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1000, name=NORMAL_CLOSURE, reason=None>
2020-05-15 02:02:41.267 | INFO     | legimens.App:_handler:74 - New ws connection of /Legi_0x7f303b29c9a0 from ::1
2020-05-15 02:02:41.269 | INFO     | legimens.App:_handle_obj_ref:105 - Yield initial update {"value": []}


In [12]:
li.append(len(li))

In [15]:
li.pop()

2

Values should be instantly updated in x variable